In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### CKD 새로 생성
- baseline 1기, 2-7기 사이에 연달아서 2번 있으면 CKD = 1, 아니면 CKD = 0    
- (2021.11.23) TIME 변수 추가

In [3]:
final = pd.read_csv('data\\MME_.csv', encoding = 'euc-kr', low_memory = False)
final

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,1.575,48.0,63.0,91.0,16.0,...,3.50,1.75,13400.0,2571.0,2,96.0,63.0,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.755,75.0,81.3,88.0,27.0,...,5.50,0.00,15700.0,2409.0,2,95.0,68.0,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,1.668,50.4,74.0,82.0,52.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,1.556,53.6,76.4,110.0,38.0,...,0.75,0.75,14700.0,886.5,2,102.0,74.0,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,1.745,88.0,94.5,81.0,5.0,...,5.50,0.00,19500.0,2409.0,2,131.0,84.0,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,1.641,60.9,74.0,88.0,NaN,...,NaN,NaN,17900.0,NaN,0,92.0,62.0,74.201919,22.615184,1
70206,A07,201407.0,NIH1604999732,75.0,1,1.632,50.8,81.3,110.0,NaN,...,NaN,NaN,10500.0,NaN,0,131.0,66.0,68.312024,19.073193,3
70207,A07,201409.0,NIH1604999772,55.0,1,1.635,66.2,88.3,85.0,NaN,...,NaN,NaN,13900.0,NaN,0,NaN,NaN,50.048887,24.764096,2
70208,A07,201403.0,NIH1604999929,53.0,2,1.530,63.2,85.3,95.0,NaN,...,NaN,NaN,27100.0,NaN,0,112.0,71.0,74.230720,26.998163,2


In [4]:
new_ckd = final[['기수', 'NIHID', 'AGE', 'eGFR']]
new_ckd

,기수,NIHID,AGE,eGFR
0,A01,NIH1604000095,43.0,78.564168
1,A01,NIH1604000171,42.0,92.420166
2,A01,NIH1604000338,69.0,86.839489
3,A01,NIH1604000362,48.0,102.784214
4,A01,NIH1604000424,47.0,112.380813
...,...,...,...,...
70205,A07,NIH1604999643,55.0,74.201919
70206,A07,NIH1604999732,75.0,68.312024
70207,A07,NIH1604999772,55.0,50.048887
70208,A07,NIH1604999929,53.0,74.230720


In [5]:
unique_id = new_ckd['NIHID'].unique().tolist()

In [6]:
# ID별로 정렬

finalDf = pd.DataFrame()
for id in unique_id: 
    df = pd.DataFrame(new_ckd.loc[new_ckd['NIHID'] == id])
    finalDf = pd.concat([finalDf, df], axis = 0)

In [7]:
finalDf.reset_index(inplace = True, drop = True)
finalDf

,기수,NIHID,AGE,eGFR
0,A01,NIH1604000095,43.0,78.564168
1,A02,NIH1604000095,45.0,NaN
2,A03,NIH1604000095,47.0,59.931748
3,A04,NIH1604000095,NaN,NaN
4,A05,NIH1604000095,51.0,56.406072
...,...,...,...,...
70205,A03,NIH1604999956,NaN,NaN
70206,A04,NIH1604999956,NaN,NaN
70207,A05,NIH1604999956,NaN,NaN
70208,A06,NIH1604999956,NaN,NaN


In [8]:
finalDf.loc[finalDf['eGFR'] <= 60, 'CKD'] = 1
finalDf.loc[finalDf['eGFR'] > 60, 'CKD'] = 0

In [9]:
finalDf.head(50)

,기수,NIHID,AGE,eGFR,CKD
0,A01,NIH1604000095,43.0,78.564168,0.0
1,A02,NIH1604000095,45.0,NaN,NaN
2,A03,NIH1604000095,47.0,59.931748,1.0
3,A04,NIH1604000095,NaN,NaN,NaN
4,A05,NIH1604000095,51.0,56.406072,1.0
5,A06,NIH1604000095,53.0,56.220099,1.0
6,A07,NIH1604000095,55.0,47.620724,1.0
7,A01,NIH1604000171,42.0,92.420166,0.0
8,A02,NIH1604000171,44.0,NaN,NaN
9,A03,NIH1604000171,46.0,99.390358,0.0


In [10]:
# Nan 그냥 다 빼고

finalDf.dropna(inplace = True)

In [11]:
finalDf.reset_index(inplace = True, drop = True)
finalDf.head(50)

,기수,NIHID,AGE,eGFR,CKD
0,A01,NIH1604000095,43.0,78.564168,0.0
1,A03,NIH1604000095,47.0,59.931748,1.0
2,A05,NIH1604000095,51.0,56.406072,1.0
3,A06,NIH1604000095,53.0,56.220099,1.0
4,A07,NIH1604000095,55.0,47.620724,1.0
5,A01,NIH1604000171,42.0,92.420166,0.0
6,A03,NIH1604000171,46.0,99.390358,0.0
7,A04,NIH1604000171,48.0,98.003763,0.0
8,A05,NIH1604000171,50.0,96.636512,0.0
9,A06,NIH1604000171,52.0,90.509321,0.0


In [15]:
# lst = ['A02', 'A03', 'A04', 'A05', 'A06', 'A07']

In [12]:
# test 인듯?
df = pd.DataFrame(finalDf.loc[finalDf['NIHID'] == 'NIH1604000338'].reset_index(drop = True))
df['final_CKD'] = 0
for i in range(len(df) - 1): 
    if (df['CKD'][i] == 1) & (df['CKD'][i + 1] == 1): 
        df['final_CKD'] = 1

In [13]:
df

,기수,NIHID,AGE,eGFR,CKD,final_CKD
0,A01,NIH1604000338,69.0,86.839489,0.0,1
1,A02,NIH1604000338,71.0,60.473461,0.0,1
2,A03,NIH1604000338,73.0,63.444719,0.0,1
3,A04,NIH1604000338,75.0,59.395791,1.0,1
4,A05,NIH1604000338,77.0,65.853842,0.0,1
5,A06,NIH1604000338,79.0,57.168765,1.0,1
6,A07,NIH1604000338,81.0,53.656610,1.0,1


In [18]:
# fianl_CKD 생성

ckd_df = pd.DataFrame()
for id in unique_id: 
    df = pd.DataFrame(finalDf.loc[finalDf['NIHID'] == id].reset_index(drop = True))
    first_age = df['AGE'][df['기수'] == 'A01']
    df['final_CKD'] = 0

    for i in range(len(df) - 1, -1, -1):   # last exam age 찾기
        if (pd.isnull(df['AGE'][i]) == False): 
            last_exam_age = df['AGE'][i]
            break; 
            
    for i in range(1, len(df) - 1): 
        if (df['CKD'][i] == 1) & (df['CKD'][i + 1] == 1): 
            df['final_CKD'] = 1
            diagnosed_age = df['AGE'][i]
    
    if df['final_CKD'][0] == 0:    # 진단 받지 않은 사람이라면
        time_l = last_exam_age - first_age  # TIME은 마지막 검진 나이 - 1차 검진 나이
    if df['final_CKD'][0] == 1:    # 진단 받은 사람이라면
        time_1 = diagnosed_age - first_age  # TIME은 진단 나이 - 1차 검진 나이

    df['TIME'] = time_l    
    ckd_df = pd.concat([ckd_df, df], axis = 0)
    ckd_df.reset_index(drop = True, inplace = True)

In [20]:
ckd_df

,기수,NIHID,AGE,eGFR,CKD,final_CKD,TIME
0,A01,NIH1604000095,43.0,78.564168,0.0,1,0.0
1,A03,NIH1604000095,47.0,59.931748,1.0,1,NaN
2,A05,NIH1604000095,51.0,56.406072,1.0,1,NaN
3,A06,NIH1604000095,53.0,56.220099,1.0,1,NaN
4,A07,NIH1604000095,55.0,47.620724,1.0,1,NaN
...,...,...,...,...,...,...,...
49206,A01,NIH1604999929,41.0,107.964675,0.0,0,12.0
49207,A02,NIH1604999929,43.0,90.587425,0.0,0,NaN
49208,A06,NIH1604999929,51.0,80.731689,0.0,0,NaN
49209,A07,NIH1604999929,53.0,74.230720,0.0,0,NaN


In [28]:
k = ckd_df.copy()

In [29]:
k.drop_duplicates(inplace = True, subset = ['NIHID'])

In [30]:
k.drop(['기수', 'AGE', 'eGFR', 'CKD'], axis = 1, inplace = True)
k.reset_index(drop = True, inplace = True)
k

,NIHID,final_CKD,TIME
0,NIH1604000095,1,0.0
1,NIH1604000171,0,12.0
2,NIH1604000338,1,12.0
3,NIH1604000362,0,0.0
4,NIH1604000424,0,6.0
...,...,...,...
10025,NIH1604999643,0,12.0
10026,NIH1604999732,0,12.0
10027,NIH1604999772,1,12.0
10028,NIH1604999929,0,12.0


In [31]:
k['final_CKD'].value_counts()

0    9218
1     812
Name: final_CKD, dtype: int64

In [32]:
k['TIME'].value_counts()

12.0    4100
11.0    1487
0.0     1306
10.0     624
2.0      494
4.0      488
8.0      445
6.0      314
9.0      223
13.0     157
3.0      115
1.0       97
5.0       96
7.0       71
14.0       6
15.0       3
17.0       2
Name: TIME, dtype: int64

In [33]:
k.to_csv('data\\CKD_final_TIMEadded_Baseline1st+OccuredTwiceOrMoreInARow.csv', encoding = 'euc-kr', index = False)